In [21]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.gridspec as gridspec
import cantera as ct
from sklearn.linear_model import LinearRegression
import os
import concurrent.futures

def reverse_Arrhenius(a):   
    reaction_idx=a
    gas = ct.Solution(cti_file, 'gas')
    surf = ct.Interface(cti_file,'surface1', [gas])

    t=np.linspace(100,800, 50)
    k_forward=np.zeros(len(t))
    k_reverse=np.zeros(len(t))
    for i in range(len(t)):
      gas.TPX =t[i]+273, ct.one_atm, 'Ar:73.9695'
      surf.TP = t[i]+273, ct.one_atm
      surf.coverages = {'site(100)':1}
      k_reverse[i]=surf.reverse_rate_constants[reaction_idx]
      k_forward[i]=surf.forward_rate_constants[reaction_idx]
   
    T_reciprok=1/(t+273)*1000    
    log_k_forward=np.log(k_forward)
    log_k_reverse=np.log(k_reverse)
    log_k_reverse=log_k_reverse.reshape(len(k_reverse),1)
    T_reciprok=T_reciprok.reshape(len(T_reciprok),1)

    #Linear regression of an Arrhenius expression to the calculated rate constant
    model =LinearRegression(fit_intercept=True)
    model.fit(T_reciprok,log_k_reverse)
    yfit=model.predict(log_k_reverse)
    Ea_R=-float(model.coef_[0])
    ln_k0=float(model.intercept_[0])
    activation_barrier=np.round(Ea_R*8.314,3)
    pre_exponential=float(np.format_float_scientific(np.exp(ln_k0)*10,precision=3))
    #print(surf.reaction_equations()[a])
    #print(pre_exponential,activation_barrier)
    return pre_exponential, activation_barrier 

def get_Arrhenius(phase, index):
    """Use this subroutine to update the Arrhenius coefficients for a reaction, given by index. 
    It takes as input:
    (i) an index for the reaction of interest"""
    # create a copy of the reaction
    my_rxn = phase.reaction(index)
    A = my_rxn.rate.pre_exponential_factor*10
    b = my_rxn.rate.temperature_exponent
    Ea = np.round(my_rxn.rate.activation_energy/1e6/4.184,3)
    #print(A,Ea)
    return A,b,Ea

def get_nasa(phase, index):
        my_species=phase.species(index)
        old_coeffs=my_species.thermo.coeffs
        coeff_6=(np.format_float_scientific(old_coeffs[6],precision=8))
        coeff_7=old_coeffs[7]
        thermo="".join((coeff_6,', ', str(coeff_7),']'))
        return thermo
    



cti_file="".join(('Ni100.yaml'))
new_file="".join(('Ni100_covdep.yaml'))
file_copy="".join(('cp ',cti_file,' ', new_file))
os.popen(file_copy)
gas = ct.Solution(cti_file, 'gas')
gas.TPX = 373, ct.one_atm, 'Ar:73.9695'

# import the surface model
surf = ct.Interface(cti_file,'surface1', [gas])
surf.TP = 273, ct.one_atm
surf.coverages = {'site(100)':1}

idx=[]
for i in range(surf.n_reactions):
        #Check for all reactions where either CO* is a product
        if 'OC(100)' in surf.products(i):
                idx.append(i) 
                string="".join((surf.products(i),' <=> ', surf.reactants(i)))
        
line_idx=0               
for i in idx:
        params=get_Arrhenius(surf,i)
        old_rate="".join(('rate-constant: {A: ', str(params[0]),', b: 0.0, Ea: ',str(params[2]),'}'))
        rev_params=reverse_Arrhenius(i)
        new_rate="".join(('rate-constant: {A: ', str(rev_params[0]),', b: 0.0, Ea: ',str(rev_params[1]),'} #reverse'))

        name="".join(("  # Reaction ",str(i+1)))
        #print(name)
        input_file = open(new_file,'r')
        lines = input_file.readlines()
        input_file.close()
        new_file_content="" 

        switch=False
        for index,line in enumerate(lines):
            #print("Line {}: {}".format(index, line.strip()))
            if line.strip().endswith(name):
                 old=line.strip()
                 modified="".join(("- equation: ", surf.products(i), " <=> ", surf.reactants(i),"  # Reaction ",str(i+1)))
                 new_line=line.replace(old,modified)
                 new_file_content += new_line 
                 switch=True
                 line_idx=index
            elif index == line_idx+1:
                if switch==True:
                    print(line_idx)
                    old=line.strip()
                    modified=new_rate
                    new_line=line.replace(old,modified)
                    new_file_content += new_line 
                else:
                     old=line.strip()
                     new_line=line.replace(old,old)
                     new_file_content += new_line 
            else:
                 switch=False
                 old=line.strip()
                 new_line=line.replace(old,old)
                 new_file_content += new_line 
        
        writing_file = open(new_file, "w")
        writing_file.write(new_file_content)
        writing_file.close()
        
#writing_file = open(new_file, "w")
#writing_file.write(new_file_content)
#writing_file.close()




206
208
210
